In [1]:
!cd ~/Developer/kuzu/ &&  make clean-python-api && make release NUM_THREADS=20

rm -rf tools/python_api/build
cd external && \
	mkdir -p build && \
	cd build && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
	cmake --build . --config Release -- -j 20
-- Configuring done (0.0s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/lc/Developer/kuzu/external/build
[100%] Built target apache_arrow
mkdir -p build/release && \
	cd build/release && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
	cmake --build . --config Release -- -j 20
-- pybind11 v2.11.0 dev1
-- Configuring done (0.1s)
-- Generating done (0.3s)
-- Build files have been written to: /Users/lc/Developer/kuzu/build/release
[  0%] Built target kuzu_transaction
[  1%] Built target utf8proc
[  1%] Built target kuzu_common_copier_config
[  1%] Built target kuzu_common_vector
[  2%] Built target kuzu_common_arrow
[  3%] Built target kuzu_common_task_system
[  3%] Built target kuzu_common_data_chunk
[  3%] Built target kuzu_catalog
[  4%] Built target kuzu_binder_expression
[  5%] Built targ

In [2]:
import sys
sys.path.append('/Users/lc/Developer/kuzu/tools/python_api/build')
import kuzu
import torch
import time
from torch_geometric.loader import NeighborLoader

In [3]:
DB_PATH = '/Users/lc/Developer/kuzu_ogb_mag'
db = kuzu.Database(DB_PATH, buffer_pool_size=1024**3)
fs, gs = db.get_torch_geometric_remote_backend(1)

In [4]:
gs.get_edge_index(
        ('paper', 'cites', 'paper'), layout='coo', is_sorted=False)

tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
        [    88,  27449, 121051,  ..., 707740, 732008, 732389]])

In [5]:
fs['paper', 'year', 100]

tensor([2013])

In [6]:
loader_kuzu = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [30] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, 736389)),
    input_time=None
)

/opt/homebrew/lib/python3.11/site-packages/torch_geometric/data/graph_store.py:360: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if (src[0].storage().data_ptr() == dst[1].storage().data_ptr()
/opt/homebrew/lib/python3.11/site-packages/torch_geometric/data/graph_store.py:365: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  out.set_(src.storage(), storage_offset=src.storage_offset(),


In [7]:
i = 1
for b in loader_kuzu:
    i += 1
    if i == 10:
        break
start = time.time()
i = 1
for b in loader_kuzu:
    i += 1
    if i == 10:
        break
end = time.time() - start

In [8]:
i, end

(10, 6.42133092880249)

In [9]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
import torch
from torch_geometric.loader import NeighborLoader
import time

dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [10]:
loader_pyg = NeighborLoader(
    data=data,
    num_neighbors={key: [30] * 2 for key in data.edge_types},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, 736389)),
    input_time=None,
    num_workers=20,
    filter_per_worker=True,
    persistent_workers=True
)

In [11]:
i = 1
for b in loader_pyg:
    i += 1
    if i == 100:
        break
start = time.time()
i = 1
for b in loader_pyg:
    i += 1
    if i == 100:
        break
end = time.time() - start

/opt/homebrew/lib/python3.11/site-packages/torch_geometric/loader/utils.py:53: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = value.storage()._new_shared(numel)
/opt/homebrew/lib/python3.11/site-packages/torch_geometric/loader/utils.py:53: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = value.storage()._new_shared(numel)
/opt/homebrew/lib/python3.11/site-packages/torch_geometric/loader/utils.py:53: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will b

In [12]:
i, end

(100, 0.9346771240234375)